In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Dropout, GlobalAveragePooling2D, UpSampling2D, Input, LeakyReLU
from keras.layers import Conv2DTranspose
from tensorflow.keras.layers import MaxPooling2D, BatchNormalization
from tensorflow.keras.utils import Sequence
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.inception_v3 import InceptionV3
from sklearn.neural_network import MLPClassifier
from keras.utils.vis_utils import plot_model
from keras.layers.core import Dense, Activation, Dropout
from tensorflow.keras.applications.resnet50 import preprocess_input
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import confusion_matrix
import keras
from keras import layers
import random as rnd
import cv2
import matplotlib.image as mpimg
from PIL import Image
%matplotlib inline

# Creating DataFrame

In [2]:
main_df = pd.DataFrame()
main_path = '../input/grapevine-leaves-image-dataset/Grapevine_Leaves_Image_Dataset/'
path_Ak = main_path + 'Ak'
path_Ala_Idris = main_path + 'Ala_Idris'
path_Buzgulu = main_path + 'Buzgulu'
path_Dimnit = main_path + 'Dimnit'
path_Nazli = main_path + 'Nazli'

main_df['images'] = os.listdir(path_Ak) + os.listdir(path_Ala_Idris) + os.listdir(path_Buzgulu) + os.listdir(path_Dimnit) + os.listdir(path_Nazli)

classes = []
paths = []
for image in main_df['images']:
    class_ = image.split(' (')[0]
    classes.append(class_)
    paths.append(main_path+class_+'/'+image)

main_df['classes'] = classes
main_df['path'] = paths
print(len(main_df))
main_df.head()

In [3]:
X_train1, X_test, y_train1, y_test = train_test_split(main_df[['path', 'classes']], 
                                                  main_df[['classes']], 
                                                  test_size=0.2, 
                                                  random_state=101)
X_train, X_val, y_train, y_val = train_test_split(X_train1[['path', 'classes']], 
                                                  X_train1[['classes']], 
                                                  test_size=0.2, 
                                                  random_state=101)
print(len(X_train), len(X_val), len(X_test))

In [4]:
plt.figure(figsize = (15,12))
for idx,i in enumerate(main_df.classes.unique()):
    plt.subplot(4,7,idx+1)
    df = main_df[main_df['classes'] ==i].reset_index(drop = True)
    image_path = df.loc[rnd.randint(0, len(df))-1,'path']
    img = Image.open(image_path)
    print(img.size)
    img = img.resize((224,224))
    plt.imshow(img)
    plt.axis('off')
    plt.title(i)
plt.tight_layout()
plt.show()

# Random Seed

In [6]:
main_path = '../input/grapevine-leaves-image-dataset/Grapevine_Leaves_Image_Dataset/'
labels = ['Ak', 'Ala_Idris', 'Buzgulu', 'Dimnit', 'Nazli']
Ak_route = main_path + 'Ak'
Ala_Idris_route = main_path + 'Ala_Idris'
Buzgulu_route = main_path + 'Buzgulu'
Dimnit_route = main_path + 'Dimnit'
Nazli_route = main_path + 'Nazli'
count_bar = {}
count_bar['labels'] = labels
count_bar['count'] = [len(os.listdir(main_path+i)) for i in labels]
count_bar_df = pd.DataFrame(count_bar, index=[1,2,3,4,5])

root_dir = "./grape_dataset"
#import shutil
#shutil.rmtree('./grape_dataset')
    
path = os.path.join(root_dir, "training")
os.makedirs(path)
for i in labels:
    os.makedirs(os.path.join(path,i))
path = os.path.join(root_dir, "validating")
os.makedirs(path)
for i in labels:
    os.makedirs(os.path.join(path,i))
path = os.path.join(root_dir, "testing")
os.makedirs(path)
for i in labels:
    os.makedirs(os.path.join(path,i))
def train_test_split(SOURCE, TRAINING, TESTING, VALIDATING, SPLIT_SIZE):
    files = []
    for filename in os.listdir(SOURCE):
        file = SOURCE + filename
        if os.path.getsize(file) > 0:
            files.append(filename)
        else:
            print(filename + ' is zero length, so ignoring.')
        
        import random
        from shutil import copyfile

        training_length = int((len(files) * SPLIT_SIZE)*SPLIT_SIZE)
        validating_length = int((len(files) * SPLIT_SIZE)) - training_length
        testing_length = int(len(files) * (1- SPLIT_SIZE))
        shuffled_set = random.sample(files, len(files))
        training_set = shuffled_set[0:training_length]
        validating_set = shuffled_set[training_length+1:-testing_length]
        testing_set = shuffled_set[-testing_length:]
    for filename in training_set:
        src_file = SOURCE + filename
        dest_file = TRAINING + filename
        copyfile(src_file, dest_file)
    
    for filename in validating_set:
        src_file = SOURCE + filename
        dest_file = VALIDATING + filename
        copyfile(src_file, dest_file)
        
    for filename in testing_set:
        src_file = SOURCE + filename
        dest_file = TESTING + filename
        copyfile(src_file, dest_file)
        
training_dir = "./grape_dataset/training"
testing_dir = "./grape_dataset/testing/"
validating_dir = "./grape_dataset/validating"
split_size = 0.8

from tensorflow.keras.applications.resnet50 import preprocess_input
for i in range(10):
    for i in labels:
        label_source_dir = "../input/grapevine-leaves-image-dataset/Grapevine_Leaves_Image_Dataset/" + str(i) + "/"
        training_label_dir = os.path.join(training_dir, (str(i)+"/"))
        validating_label_dir = os.path.join(validating_dir, (str(i)+"/"))
        testing_label_dir = os.path.join(testing_dir, (str(i)+"/"))
        train_test_split(label_source_dir, training_label_dir, testing_label_dir, validating_label_dir, split_size)

    resnet50_datagen = ImageDataGenerator(rotation_range=20,zoom_range=0.10,brightness_range=[0.6,1.4],channel_shift_range=0.7,width_shift_range=0.15,height_shift_range=0.15,shear_range=0.15,horizontal_flip=True,fill_mode='nearest',preprocessing_function=preprocess_input) 
    train_generator_resnet50 = resnet50_datagen.flow_from_directory(training_dir,  target_size=(227, 227),  batch_size=32,class_mode="categorical",shuffle=True,)
    val_generator_resnet50 = resnet50_datagen.flow_from_directory(validating_dir,  target_size=(227, 227),  batch_size=32,class_mode="categorical",shuffle=True,)
    resnet50_datagen_test = ImageDataGenerator(preprocessing_function=preprocess_input) 
    test_generator_resnet50 = resnet50_datagen_test.flow_from_directory(testing_dir,  target_size=(227, 227),  batch_size=32,class_mode="categorical",shuffle=True,)
    resnet = tf.keras.applications.ResNet50(input_shape=(227,227,3), include_top=False, weights='imagenet', classes=5)

    x = GlobalAveragePooling2D()(resnet.output)
    x = Dense(units=512, activation='relu')(x)
    x = Dense(units=512, activation='relu')(x)
    x = Dropout(0.5)(x)

    output = Dense(units=5,activation = 'softmax')(x)
    model_resnet = Model(resnet.input, output)



    opt = SGD(lr=0.001)
    model_resnet.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])
    savebest = tf.keras.callbacks.ModelCheckpoint('resnet_beforefinetuning.h5', save_best_only=True);
    history_resnet = model_resnet.fit(train_generator_resnet50, 
                                      validation_data=val_generator_resnet50, 
                                      epochs=35, 
                                     callbacks=[savebest])

    model_resnet.evaluate(test_generator_resnet50)[1]

In [5]:
main_path = '../input/grapevine-leaves-image-dataset/Grapevine_Leaves_Image_Dataset/'
labels = ['Ak', 'Ala_Idris', 'Buzgulu', 'Dimnit', 'Nazli']
Ak_route = main_path + 'Ak'
Ala_Idris_route = main_path + 'Ala_Idris'
Buzgulu_route = main_path + 'Buzgulu'
Dimnit_route = main_path + 'Dimnit'
Nazli_route = main_path + 'Nazli'
count_bar = {}
count_bar['labels'] = labels
count_bar['count'] = [len(os.listdir(main_path+i)) for i in labels]
print(count_bar)
count_bar_df = pd.DataFrame(count_bar, index=[1,2,3,4,5])
ax = sns.countplot(x=labels, data=count_bar_df)

In [6]:
root_dir = "./grape_dataset"
import shutil
shutil.rmtree('./grape_dataset')
    
path = os.path.join(root_dir, "training")
os.makedirs(path)
for i in labels:
    os.makedirs(os.path.join(path,i))
path = os.path.join(root_dir, "validating")
os.makedirs(path)
for i in labels:
    os.makedirs(os.path.join(path,i))
path = os.path.join(root_dir, "testing")
os.makedirs(path)
for i in labels:
    os.makedirs(os.path.join(path,i))

# Train, Validation, Test Split

In [7]:
def train_test_split(SOURCE, TRAINING, TESTING, VALIDATING, SPLIT_SIZE):
    files = []
    for filename in os.listdir(SOURCE):
        file = SOURCE + filename
        if os.path.getsize(file) > 0:
            files.append(filename)
        else:
            print(filename + ' is zero length, so ignoring.')
        
        import random
        from shutil import copyfile

        training_length = int((len(files) * SPLIT_SIZE)*SPLIT_SIZE)
        validating_length = int((len(files) * SPLIT_SIZE)) - training_length
        testing_length = int(len(files) * (1- SPLIT_SIZE))
        shuffled_set = random.sample(files, len(files))
        training_set = shuffled_set[0:training_length]
        validating_set = shuffled_set[training_length+1:-testing_length]
        testing_set = shuffled_set[-testing_length:]
    for filename in training_set:
        src_file = SOURCE + filename
        dest_file = TRAINING + filename
        copyfile(src_file, dest_file)
    
    for filename in validating_set:
        src_file = SOURCE + filename
        dest_file = VALIDATING + filename
        copyfile(src_file, dest_file)
        
    for filename in testing_set:
        src_file = SOURCE + filename
        dest_file = TESTING + filename
        copyfile(src_file, dest_file)

In [8]:
training_dir = "./grape_dataset/training"
testing_dir = "./grape_dataset/testing/"
validating_dir = "./grape_dataset/validating"
split_size = 0.8

for i in labels:
    label_source_dir = "../input/grapevine-leaves-image-dataset/Grapevine_Leaves_Image_Dataset/" + str(i) + "/"
    training_label_dir = os.path.join(training_dir, (str(i)+"/"))
    validating_label_dir = os.path.join(validating_dir, (str(i)+"/"))
    testing_label_dir = os.path.join(testing_dir, (str(i)+"/"))
    train_test_split(label_source_dir, training_label_dir, testing_label_dir, validating_label_dir, split_size)

In [9]:
os.listdir('./grape_dataset/validating/Ak')

# Data Augmentation with keras

In [10]:
def train_val_generators(TRAINING_DIR, VALIDATION_DIR):
    train_datagen = ImageDataGenerator(rescale=1.0/255.,
                                     rotation_range=40,
                                     width_shift_range=0.2,
                                     height_shift_range=0.2,
                                     zoom_range=0.1,
                                     horizontal_flip=True,
                                     vertical_flip=True,
                                     fill_mode='nearest')
    train_generator = train_datagen.flow_from_directory(directory=TRAINING_DIR,
                                                      batch_size=100,
                                                      class_mode='categorical',
                                                      target_size=(300, 300))
    validation_datagen = ImageDataGenerator(rescale=1.0/255.,
                                     rotation_range=40,
                                     width_shift_range=0.2,
                                     height_shift_range=0.2,
                                     zoom_range=0.1,
                                     horizontal_flip=True,
                                     vertical_flip=True,
                                     fill_mode='nearest')
    validation_generator = validation_datagen.flow_from_directory(directory=VALIDATION_DIR,
                                                                batch_size=50,
                                                                class_mode='categorical',
                                                                target_size=(300, 300))

    return train_generator, validation_generator

In [11]:
train_generator, validation_generator = train_val_generators(training_dir, validating_dir)

# ResNet50

In [14]:
from tensorflow.keras.applications.resnet50 import preprocess_input
resnet50_datagen = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.10,
    brightness_range=[0.6,1.4],
    channel_shift_range=0.7,
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode='nearest',
    preprocessing_function=preprocess_input
) 
train_generator_resnet50 = resnet50_datagen.flow_from_directory(
        training_dir,  
        target_size=(227, 227),  
        batch_size=32,
        class_mode="categorical",
        shuffle=True,
)
val_generator_resnet50 = resnet50_datagen.flow_from_directory(
        validating_dir,  
        target_size=(227, 227),  
        batch_size=32,
        class_mode="categorical",
        shuffle=True,
)
resnet50_datagen_test = ImageDataGenerator(
    preprocessing_function=preprocess_input
) 
test_generator_resnet50 = resnet50_datagen_test.flow_from_directory(
        testing_dir,  
        target_size=(227, 227),  
        batch_size=32,
        class_mode="categorical",
        shuffle=True,
)

In [15]:
resnet = tf.keras.applications.ResNet50(input_shape=(227,227,3), include_top=False, weights='imagenet', classes=5)

x = GlobalAveragePooling2D()(resnet.output)
x = Dense(units=512, activation='relu')(x)
x = Dense(units=512, activation='relu')(x)
x = Dropout(0.5)(x)

output = Dense(units=5,activation = 'softmax')(x)
model_resnet = Model(resnet.input, output)
model_resnet.summary()

In [16]:
#resnet_learning_rate = 0.001
opt = SGD(lr=0.001)
model_resnet.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])
savebest = tf.keras.callbacks.ModelCheckpoint('resnet_beforefinetuning.h5', save_best_only=True);
history_resnet = model_resnet.fit(train_generator_resnet50, 
                                  validation_data=val_generator_resnet50, 
                                  epochs=35, 
                                 callbacks=[savebest])

In [17]:
model_resnet.evaluate(test_generator_resnet50)[1]

In [18]:
f,ax=plt.subplots(2,1,figsize=(10,10)) 
#Assigning the first subplot to graph training loss and validation loss
ax[0].plot(history_resnet.history['loss'],label='Training Loss')
ax[0].plot(history_resnet.history['val_loss'],label='Validation Loss')

#Plotting the training accuracy and validation accuracy
ax[1].plot(history_resnet.history['accuracy'],label='Training Accuracy')
ax[1].plot(history_resnet.history['val_accuracy'],label='Validation Accuracy')

plt.legend()

In [19]:
from sklearn.metrics import confusion_matrix
num_of_test_samples = 100
batch_size = 32
Y_pred_res = model_resnet.predict_generator(test_generator_resnet50, num_of_test_samples // batch_size+1)
y_pred_res = np.argmax(Y_pred_res, axis=1)
print('Confusion Matrix')
conf_matrix_res = confusion_matrix(test_generator_resnet50.classes, y_pred_res)
cm_res = np.array2string(conf_matrix_res)
print(conf_matrix_res)

In [20]:
resnet50_datagen = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.10,
    brightness_range=[0.6,1.4],
    channel_shift_range=0.7,
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode='nearest',
    preprocessing_function=preprocess_input
) 


kf = KFold(n_splits = 10)
kfold = StratifiedKFold(n_splits=10,shuffle=True,random_state=42)
results = []

Y = X_train1[['classes']]
train_x = X_train1.drop(['classes'],axis=1)

for train_index, val_index in kf.split(np.zeros(400),Y):
    training_data = X_train1.iloc[train_index]
    validation_data = X_train1.iloc[val_index]
    training_set = resnet50_datagen.flow_from_dataframe(dataframe = training_data, 
                                                      x_col="path", 
                                                      y_col="classes",
                                                      class_mode="categorical",
                                                      target_size=(227, 227), 
                                                      batch_size=32)
    validation_set = resnet50_datagen.flow_from_dataframe(dataframe = validation_data, 
                                                        x_col = "path", 
                                                        y_col = "classes",
                                                        class_mode = "categorical",
                                                        target_size = (227, 227), 
                                                        batch_size = 32)
    resnet = tf.keras.applications.ResNet50(input_shape=(227,227,3), include_top=False, weights='imagenet', classes=5)
    x = GlobalAveragePooling2D()(resnet.output)
    x = Dense(units=512, activation='relu')(x)
    x = Dense(units=512, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(units=5,activation = 'softmax')(x)
    model_resnet = Model(resnet.input, output)
    
    opt = SGD(lr=0.001)
    model_resnet.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])
    history_resnet = model_resnet.fit(training_set, validation_data=validation_set, epochs=30)

    resu = model_resnet.evaluate(test_generator_resnet50)
    results.append(resu)
    print(resu)
print(results)

# Inception_V3

In [12]:
from tensorflow.keras.applications.inception_v3 import preprocess_input
inception_v3_datagen = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.10, 
    brightness_range=[0.6,1.4],
    channel_shift_range=0.7,
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode='nearest',
    preprocessing_function=preprocess_input
)
train_generator_inception_v3 = inception_v3_datagen.flow_from_directory(
        training_dir,  
        target_size=(300, 300),  
        batch_size=32,
        class_mode="categorical",
        shuffle=True,
)
val_generator_inception_v3 = inception_v3_datagen.flow_from_directory(
        validating_dir,  
        target_size=(300, 300),  
        batch_size=32,
        class_mode="categorical",
        shuffle=True,
)
inception_v3_datagen_test = ImageDataGenerator(
    preprocessing_function=preprocess_input
)
test_generator_inception_v3 = inception_v3_datagen_test.flow_from_directory(
        testing_dir,  
        target_size=(300, 300),  
        batch_size=32,
        class_mode="categorical",
        shuffle=True,
)

In [13]:
InceptionV3_model = tf.keras.applications.InceptionV3(weights='imagenet', include_top=False, input_shape=(300, 300, 3))

for layer in InceptionV3_model.layers[:-15]:
    layer.trainable = False

x = InceptionV3_model.output
x = GlobalAveragePooling2D()(x)
x = Flatten()(x)
x = Dense(units=512, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(units=512, activation='relu')(x)
x = Dropout(0.3)(x)
output  = Dense(units=5, activation='softmax')(x)
model_inception = Model(InceptionV3_model.input, output)

model_inception.summary()

In [14]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model_inception.compile(
        optimizer=optimizer,
        loss="categorical_crossentropy",
        metrics=["accuracy"])
history_inception = model_inception.fit(train_generator_inception_v3, steps_per_epoch=4, epochs=25, validation_data=val_generator_inception_v3, validation_steps=2)

In [15]:
model_inception.evaluate(test_generator_inception_v3)[1]

In [16]:
f,ax=plt.subplots(2,1,figsize=(10,10)) 

ax[0].plot(history_inception.history['loss'],label='Training Loss')
ax[0].plot(history_inception.history['val_loss'],label='Validation Loss')

ax[1].plot(history_inception.history['accuracy'],label='Training Accuracy')
ax[1].plot(history_inception.history['val_accuracy'],label='Validation Accuracy')
plt.legend()

In [17]:
num_of_test_samples = 100
batch_size = 32
Y_pred_inc = model_inception.predict_generator(test_generator_inception_v3, num_of_test_samples // batch_size+1)
y_pred_inc = np.argmax(Y_pred_inc, axis=1)
print('Confusion Matrix')
conf_matrix_inc = confusion_matrix(test_generator_inception_v3.classes, y_pred_inc)
cm_inc = np.array2string(conf_matrix_inc)
print(conf_matrix_inc)

In [18]:
from statistics import mean
inception_v3_datagen = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.10, 
    brightness_range=[0.6,1.4],
    channel_shift_range=0.7,
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode='nearest',
    preprocessing_function=preprocess_input
)
inception_v3_datagen_test = ImageDataGenerator(
    preprocessing_function=preprocess_input
) 
kf = KFold(n_splits = 10)
kfold = StratifiedKFold(n_splits=10,shuffle=True,random_state=42)
results_inc = []

Y = X_train1[['classes']]

for train_index, val_index in kf.split(np.zeros(400),Y):
    training_data = X_train1.iloc[train_index]
    validation_data = X_train1.iloc[val_index]
    training_set = inception_v3_datagen.flow_from_dataframe(dataframe = training_data, 
                                                      x_col="path", 
                                                      y_col="classes",
                                                      class_mode="categorical",
                                                      target_size=(227, 227), 
                                                      batch_size=32)
    validation_set = inception_v3_datagen.flow_from_dataframe(dataframe = validation_data, 
                                                        x_col = "path", 
                                                        y_col = "classes",
                                                        class_mode = "categorical",
                                                        target_size = (227, 227), 
                                                        batch_size = 32)
    
    InceptionV3_model = tf.keras.applications.InceptionV3(weights='imagenet', include_top=False, input_shape=(300, 300, 3))
    for layer in InceptionV3_model.layers[:-15]:
        layer.trainable = False
    x = InceptionV3_model.output
    x = GlobalAveragePooling2D()(x)
    x = Flatten()(x)
    x = Dense(units=512, activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(units=512, activation='relu')(x)
    x = Dropout(0.3)(x)
    output  = Dense(units=5, activation='softmax')(x)
    model_inception = Model(InceptionV3_model.input, output)

    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
    model_inception.compile(optimizer=optimizer, loss="categorical_crossentropy",metrics=["accuracy"])
    history_inception = model_inception.fit(train_generator_inception_v3, steps_per_epoch=4, epochs=25, validation_data=val_generator_inception_v3, validation_steps=2)
    
    resu = model_inception.evaluate(test_generator_inception_v3)
    results_inc.append(resu)
    print(resu)
print(results_inc)

# EfficientNet

In [19]:
from tensorflow.keras.applications.efficientnet import preprocess_input
EfficientNetB3_datagen = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.10,
    brightness_range=[0.6,1.4],
    channel_shift_range=0.7,
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode='nearest',
    preprocessing_function=preprocess_input
) 
train_generator_EfficientNetB3 = EfficientNetB3_datagen.flow_from_directory(
        training_dir,  
        target_size=(300, 300), 
        batch_size=32,
        class_mode="categorical",
        shuffle=True,
)
val_generator_EfficientNetB3 = EfficientNetB3_datagen.flow_from_directory(
        validating_dir,  
        target_size=(300, 300), 
        batch_size=32,
        class_mode="categorical",
        shuffle=True,
)
EfficientNetB3_datagen_test = ImageDataGenerator(
    preprocessing_function=preprocess_input
) 
test_generator_EfficientNetB3 = EfficientNetB3_datagen_test.flow_from_directory(
        testing_dir,  
        target_size=(300, 300), 
        batch_size=32,
        class_mode="categorical",
        shuffle=True,
)

In [20]:
img_shape=(300, 300, 3)
model_name='EfficientNetB3'
EfficientNetB3_model=tf.keras.applications.efficientnet.EfficientNetB3(include_top=False, weights="imagenet",input_shape=img_shape, pooling='max') 
for layer in EfficientNetB3_model.layers[:-15]:
    layer.trainable = False
x = EfficientNetB3_model.output
x = BatchNormalization(axis=-1, epsilon=0.001)(x)
x = Dense(256,activation='relu')(x)
x = Dropout(rate=.3)(x)       
output=Dense(5, activation='softmax')(x)
EfficientNetB3_model = Model(inputs=EfficientNetB3_model.input, outputs=output)

In [21]:
EfficientNetB3_model.compile('Adam', loss='categorical_crossentropy', metrics=['accuracy']) 
history_eff = EfficientNetB3_model.fit(train_generator_EfficientNetB3, epochs=25, validation_data=val_generator_EfficientNetB3)

In [22]:
EfficientNetB3_model.evaluate(test_generator_EfficientNetB3)[1]

In [23]:
f,ax=plt.subplots(2,1,figsize=(10,10)) 

ax[0].plot(history_eff.history['loss'],label='Training Loss')
ax[0].plot(history_eff.history['val_loss'],label='Validation Loss')

ax[1].plot(history_eff.history['accuracy'],label='Training Accuracy')
ax[1].plot(history_eff.history['val_accuracy'],label='Validation Accuracy')
plt.legend()

In [24]:
num_of_test_samples = 100
batch_size = 32
Y_pred_eff = EfficientNetB3_model.predict_generator(test_generator_EfficientNetB3, num_of_test_samples // batch_size+1)
y_pred_eff = np.argmax(Y_pred_eff, axis=1)
print('Confusion Matrix')
conf_matrix_eff = confusion_matrix(test_generator_EfficientNetB3.classes, y_pred_eff)
cm_inc = np.array2string(conf_matrix_eff)
print(conf_matrix_eff)

In [25]:
from statistics import mean
EfficientNetB3_datagen = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.10,
    brightness_range=[0.6,1.4],
    channel_shift_range=0.7,
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode='nearest',
    preprocessing_function=preprocess_input
) 

kf = KFold(n_splits = 10)
kfold = StratifiedKFold(n_splits=10,shuffle=True,random_state=42)
results_eff = []

Y = X_train1[['classes']]

for train_index, val_index in kf.split(np.zeros(400),Y):
    training_data = X_train1.iloc[train_index]
    validation_data = X_train1.iloc[val_index]
    training_set = EfficientNetB3_datagen.flow_from_dataframe(dataframe = training_data, 
                                                      x_col="path", 
                                                      y_col="classes",
                                                      class_mode="categorical",
                                                      target_size=(300, 300), 
                                                      batch_size=32)
    validation_set = EfficientNetB3_datagen.flow_from_dataframe(dataframe = validation_data, 
                                                        x_col = "path", 
                                                        y_col = "classes",
                                                        class_mode = "categorical",
                                                        target_size = (300, 300), 
                                                        batch_size = 32)
        
    
    img_shape=(300, 300, 3)
    model_name='EfficientNetB3'
    EfficientNetB3_model=tf.keras.applications.efficientnet.EfficientNetB3(include_top=False, weights="imagenet",input_shape=img_shape, pooling='max') 
    for layer in EfficientNetB3_model.layers[:-15]:
        layer.trainable = False
    x = EfficientNetB3_model.output
    x = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(x)
    x = Dense(256,activation='relu')(x)
    x = Dropout(rate=.4, seed=123)(x)       
    output=Dense(5, activation='softmax')(x)
    EfficientNetB3_model = Model(inputs=EfficientNetB3_model.input, outputs=output)
    
    EfficientNetB3_model.compile('Adam', loss='categorical_crossentropy', metrics=['accuracy']) 
    history_eff = EfficientNetB3_model.fit(train_generator_EfficientNetB3, epochs=25, validation_data=val_generator_EfficientNetB3)
    
    resu = EfficientNetB3_model.evaluate(test_generator_EfficientNetB3)
    results_eff.append(resu)
    print(resu)
print(results_eff)

# Autoencoder

In [30]:
class DataGenerator(Sequence):
    def __init__(self, df_file, base_dir, output_size, shuffle=False, batch_size=10):
        self.df = df_file
        self.base_dir = base_dir
        self.output_size = output_size
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.on_epoch_end()
        self.flag = 0
        self.y_mlp = []
        self.X_mlp = []
    def on_epoch_end(self):
        self.indices = np.arange(len(self.df))
        if self.shuffle:
            np.random.shuffle(self.indices)
    def __len__(self):
        return int(len(self.df) / self.batch_size)
    def __getitem__(self, idx):
        X = np.empty((self.batch_size, *self.output_size, 4))
        y = x
        indices = self.indices[idx*self.batch_size:(idx+1)*self.batch_size]
        for i, data_index in enumerate(indices):
            img_path = self.df.iloc[data_index, 0]
            img = mpimg.imread(img_path)
            img2 = cv2.resize(img, self.output_size)
            X[i,] = img2
            self.X_mlp.append(img2)
            self.y_mlp.append(self.df.iloc[data_index, 1])
        return X, X

In [31]:
train_generator_autoencoder = DataGenerator( X_train, "data", (128, 128), batch_size=8, shuffle=True)
val_generator_autoencoder = DataGenerator( X_val, "data", (128, 128), batch_size=8, shuffle=True)
test_generator_autoencoder = DataGenerator( X_test, "data", (128, 128), batch_size=8, shuffle=True)

In [32]:
len(train_generator_autoencoder.y_mlp)

In [33]:
input_img = keras.Input(shape=(128, 128, 4))

x = layers.Conv2D(256, (5, 5), activation='relu', padding='same')(input_img)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)

x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
encoded = layers.MaxPooling2D((2, 2), padding='same')(x)

encoded2 = layers.Flatten()(encoded)
encoded2 = layers.Dense(2046)(encoded2)
x = layers.Dense(16384)(encoded2)
x = layers.Reshape(target_shape=(16,16,64))(x)

x = layers.Conv2DTranspose(64, (3, 3), strides = 2, activation='relu', padding='same')(x)
x = layers.Conv2DTranspose(128, (3, 3), strides = 2, activation='relu', padding='same')(x)
x = layers.Conv2DTranspose(256, (5, 5), strides = 2, activation='relu', padding='same')(x)
decoded = layers.Conv2D(4, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = keras.Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder.summary()
plot_model(autoencoder, to_file='OwnCNN_model_plot.png', show_shapes=True, show_layer_names=True)

In [34]:
history_autoencoder = autoencoder.fit(train_generator_autoencoder,  
                                      epochs=15, 
                                      validation_data=val_generator_autoencoder)

In [35]:
encoder = keras.Model(input_img, encoded2)

class DataGenerator_mlp(Sequence):
    def __init__(self, df_file, base_dir, output_size, shuffle=False, batch_size=10):
        self.df = df_file
        self.base_dir = base_dir
        self.output_size = output_size
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.on_epoch_end()
        self.flag = 0
        self.y_mlp = []
        self.X_mlp = []
    def on_epoch_end(self):
        self.indices = np.arange(len(self.df))
        if self.shuffle:
            np.random.shuffle(self.indices)
    def __len__(self):
        return int(len(self.df) / self.batch_size)
    def __getitem__(self, idx):
        X = np.empty((self.batch_size, *self.output_size, 4))
        y = x
        indices = self.indices[idx*self.batch_size:(idx+1)*self.batch_size]
        for i, data_index in enumerate(indices):
            img_path = self.df.iloc[data_index, 0]
            img = mpimg.imread(img_path)
            img2 = cv2.resize(img, self.output_size)
            X[i,] = img2
            self.X_mlp.append(img2)
            self.y_mlp.append(self.df.iloc[data_index, 1])
        return X

In [36]:
train_generator_autoencoder_mlp = DataGenerator_mlp(X_train, "data", (128, 128), batch_size=8, shuffle=True)
val_generator_autoencoder_mlp = DataGenerator_mlp(X_val, "data", (128, 128), batch_size=8, shuffle=True)
test_generator_autoencoder_mlp = DataGenerator_mlp(X_test, "data", (128, 128), batch_size=8, shuffle=True)

In [37]:
encoder_train_prediction = encoder.predict(train_generator_autoencoder_mlp)
encoder_val_prediction = encoder.predict(val_generator_autoencoder_mlp)
encoder_test_prediction = encoder.predict(test_generator_autoencoder_mlp)
print(len(train_generator_autoencoder_mlp.y_mlp))
print(len(train_generator_autoencoder_mlp.X_mlp))
print(encoder_train_prediction.shape)
print(len(val_generator_autoencoder_mlp.y_mlp))
print(len(val_generator_autoencoder_mlp.X_mlp))
print(encoder_val_prediction.shape)
print(len(test_generator_autoencoder_mlp.y_mlp))
print(len(test_generator_autoencoder_mlp.X_mlp))
print(encoder_test_prediction.shape)

In [20]:
wer = mpimg.imread(X_train['path'][0])
wer2 = cv2.resize(wer,(512,512))
plt.imshow(autoencoder.predict(test_generator_autoencoder)[0])


In [ ]:
model_mlp = Sequential()
model_mlp.add(Dense(5000, input_dim=(128)))
model_mlp.add(Activation('relu'))
model_mlp.add(Dropout(0.15))
model_mlp.add(Dense(1000))
model_mlp.add(Activation('relu'))
model_mlp.add(Dropout(0.15))
model_mlp.add(Dense(100))
model_mlp.add(Activation('relu'))
model_mlp.add(Dropout(0.15))
model_mlp.add(Dense(5))
model_mlp.add(Activation('softmax'))
model_mlp.compile(optimizer=Adam(0.001), loss='categorical_crossentropy')
model_mlp.fit(encoder_train_prediction,train_generator_autoencoder_mlp.y_mlp[:320])

In [21]:
clf = MLPClassifier(hidden_layer_sizes = (2046), 
                    random_state=1, 
                    max_iter=300, 
                    solver = 'sgd',
                    activation = 'relu').fit(encoder_train_prediction, train_generator_autoencoder_mlp.y_mlp[:320])

In [22]:
from sklearn.metrics import classification_report,confusion_matrix
test_final_prediction = clf.predict(encoder_test_prediction)
print(classification_report(test_generator_autoencoder_mlp.y_mlp[:96], test_final_prediction))

In [38]:
input_img = keras.Input(shape=(128, 128, 4))

x = layers.Conv2D(256, (5, 5), activation='relu', padding='same')(input_img)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)

x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
encoded_second_try = layers.MaxPooling2D((2, 2), padding='same')(x)

x = layers.Conv2DTranspose(64, (3, 3), strides = 2, activation='relu', padding='same')(encoded_second_try)
x = layers.Conv2DTranspose(128, (3, 3), strides = 2, activation='relu', padding='same')(x)
x = layers.Conv2DTranspose(256, (5, 5), strides = 2, activation='relu', padding='same')(x)
decoded = layers.Conv2D(4, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder2 = keras.Model(input_img, decoded)
autoencoder2.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder2.summary()

In [39]:
encoder_second_try = keras.Model(input_img, encoded_second_try)

In [40]:
encoder_train_prediction = encoder_second_try.predict(train_generator_autoencoder_mlp)
encoder_val_prediction = encoder_second_try.predict(val_generator_autoencoder_mlp)
encoder_test_prediction = encoder_second_try.predict(test_generator_autoencoder_mlp)

In [41]:
print(len(train_generator_autoencoder_mlp.y_mlp))
print(len(train_generator_autoencoder_mlp.X_mlp))
print(encoder_train_prediction.shape)

In [52]:
inp = Input(shape=(16,16,64))

x = Conv2D(filters=32, kernel_size=(3,3),activation='relu', padding='same')(inp)
x = Conv2D(filters=32, kernel_size=(3,3),activation='relu', padding='same')(x)
x = MaxPooling2D(pool_size=2, strides=2,padding='valid')(x)

x = Conv2D(filters=64, kernel_size=(3,3),activation='relu', padding='same')(x)
x = Conv2D(filters=64, kernel_size=(3,3),activation='relu', padding='same')(x)
x = MaxPooling2D(pool_size=2, strides=2, padding='valid')(x)

x = Flatten()(x)
x = Dropout(0.4)(x)
x = Dense(units=64, activation='relu')(x)

x = Dense(units=1, activation='softmax')(x)

model_costume_cnn = Model(inp, x)
model_costume_cnn.summary()

In [ ]:
y = np.array(train_generator_autoencoder_mlp.y_mlp[:320])
model_costume_cnn.compile('Adam', loss='categorical_crossentropy', metrics=['accuracy']) 
history_eff = model_costume_cnn.fit(encoder_train_prediction, y, epochs=25)